### A/B-testing

### Part 1: Splitting

In [8]:
import hashlib
import random 
from typing import Tuple, List

In [9]:
class InvalidWeightData(ValueError):
    def __init__(self, message):
        super().__init__(message)
        self.msgfmt = message

In [136]:
class Experiment:
    """Experiment class. Contains the logic for assigning users to groups."""

    def __init__(
        self,
        experiment_id: int,
        groups: Tuple[str] = ("A", "B"),
        group_weights: List[float] = None,
    ):
        self.experiment_id = experiment_id
        self.groups = groups
        self.group_weights = group_weights
        
        # Define the salt for experiment_id.
        # The salt should be deterministic and unique for each experiment_id.
        self.salt = str(experiment_id)

        # Define the group weights if they are not provided equaly distributed
        # Check input group weights. They must be non-negative and sum to 1.
        

        if self.group_weights:
            w = 0
            for wieght in group_weights:
                if wieght < 0:
                    raise InvalidWeightData("Wieght is negative")
                w += wieght
            if w != 1:
                    raise InvalidWeightData("Sum of weights not equal to 1")
                    
    def group(self, click_id: int) -> Tuple[int, str]:
        """Assigns a click to a group.

        Parameters
        ----------
        click_id: int :
            id of the click

        Returns
        -------
        Tuple[int, str] :
            group id and group name
        """
        # Assign the click to a group randomly based on the group weights
        # Return the group id and group name
        
        click_id = str(click_id)
        click_hash = hashlib.sha256(str(click_id+self.salt).encode('utf-8')).hexdigest()
        click_hash = int(click_hash, 16)
        print(click_hash)
        if self.group_weights:
            random.seed(click_hash)
            group_ids = range(len(self.groups))
            group_id = random.choices(group_ids, self.group_weights)[0]
            
        else:
            group_id = click_hash%len(self.groups)
            
        
        return group_id, self.groups[group_id]

In [137]:
ex = Experiment(5, ('A', 'B', 'C'), [0.5, 0.05, 0.45])

In [138]:
ex.group(2)

83065501960078139073241228247522869445681663391659423428962806080829970134377


(2, 'C')

### Part-2: Statistical test

In [158]:
from scipy.stats import binom, norm, stats
from numpy import random
import numpy as np

def cpc_sample(
    n_samples: int, conversion_rate: float, reward_avg: float, reward_std: float
) -> np.ndarray:
    """Sample data."""
    
    cvr = random.binomial(n=1, p=conversion_rate, size=n_samples)
    cpa = random.norm(loc=reward_avg, scale=reward_std, size=n_samples)
    
    return cvr*cpa

In [159]:
def t_test(cpc_a: np.ndarray, cpc_b: np.ndarray, alpha=0.05
) -> Tuple[bool, float]:
    """Perform t-test.

    Parameters
    ----------
    cpc_a: np.ndarray :
        first samples    
    cpc_b: np.ndarray :
        second samples
    alpha :
         (Default value = 0.05)

    Returns
    -------
    Tuple[bool, float] :
        True if difference is significant, False otherwise
        p-value
    """
    
    _, pvalue = stats.ttest_ind(cpc_a, cpc_b)
    accept = bool(pvalue<alpha)
    return (accept, pvalue)